In [4]:
import os
from dotenv import load_dotenv
import chromadb
from chromadb.config import Settings

load_dotenv(".env")

CHROMA_SERVER_AUTH_CREDENTIALS = os.getenv("CHROMA_SERVER_AUTH_CREDENTIALS")

client = chromadb.HttpClient(
    host="https://chroma.liara.run",
    settings=Settings(
        chroma_client_auth_provider="chromadb.auth.token_authn.TokenAuthClientProvider",
        chroma_client_auth_credentials=CHROMA_SERVER_AUTH_CREDENTIALS,
    ),
)

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(".env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORG_ID = os.getenv("OPENAI_ORG_ID")

openai_client = OpenAI(
    api_key=OPENAI_API_KEY,
    organization=OPENAI_ORG_ID,
)

embedding = (
    openai_client.embeddings.create(
        input="رونالدو", model="text-embedding-3-small"
    )
    .data[0]
    .embedding
)

print(embedding)

In [ ]:
client.create_collection(name="products_openai")

In [ ]:
import pandas as pd

sample_products = pd.read_csv("../../data/sample/products.csv")
product_collection = client.get_collection(name="products_openai")

documents = []
ids = []
metadatas = []
embeddings = []

for i, row in sample_products.iterrows():
    if i <= 180: 
        continue
    print(i, row["title_fa"])
    embedding = (
        openai_client.embeddings.create(
            input=row["title_fa"], model="text-embedding-3-small"
        )
        .data[0]
        .embedding
    )
    ids.append(str(row["id"]))
    documents.append(row["title_fa"])
    embeddings.append(embedding)
    metadatas.append(row.to_dict())

In [ ]:
product_collection = client.get_collection(name="products_openai")

product_collection.upsert(
    documents=documents,
    metadatas=metadatas,
    ids=ids,
    embeddings=embeddings,
)

print(product_collection.peek(limit=5))